# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
from pandas import read_csv
weather_df = read_csv("../WeatherPy/weather_data.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,70.02,77,100,19.80,PF,1666265689
1,1,New Norfolk,-42.7826,147.0587,53.10,73,33,2.08,AU,1666265690
2,2,Arraial do Cabo,-22.9661,-42.0278,76.86,78,20,18.41,BR,1666265690
3,3,Severnoye,56.3491,78.3619,39.07,90,98,6.82,RU,1666265690
4,4,Ushuaia,-54.8000,-68.3000,33.46,86,75,20.71,AR,1666265429
...,...,...,...,...,...,...,...,...,...,...
403,403,Kuruman,-27.4524,23.4325,90.00,24,38,7.76,ZA,1666265623
404,404,Ust-Maya,60.4167,134.5333,23.23,94,100,6.49,RU,1666265750
405,405,Yako,12.9667,-2.2667,93.97,24,3,2.01,BF,1666265750
406,406,Tilichiki,60.4667,166.1000,18.91,85,91,5.86,RU,1666265630


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
weather_df = weather_df.dropna()

In [5]:
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = weather_df

# Temp between 70-80 degrees F.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] >70) & (ideal_weather_df["Max Temp"] <80)]

# Wind speed < 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,Hilo,19.7297,-155.0900,77.29,96,0,1.01,US,1666265699
98,98,Los Llanos de Aridane,28.6585,-17.9182,71.35,80,0,5.01,ES,1666265706
101,101,Ulcinj,41.9294,19.2244,72.99,44,0,5.64,ME,1666265706
144,144,Vilhena,-12.7406,-60.1458,73.62,88,0,5.75,BR,1666265711
230,230,Alice Springs,-23.7000,133.8833,71.15,40,0,6.91,AU,1666265722
261,261,Tezu,27.9167,96.1667,71.35,78,0,3.71,IN,1666265726
304,304,Buckeye,33.3703,-112.5838,75.83,47,0,0.00,US,1666265732
347,347,Yulara,-25.2406,130.9889,76.51,27,0,7.18,AU,1666265738
363,363,Palazzolo Acreide,37.0623,14.9035,71.69,41,0,4.56,IT,1666265670


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 56: Hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 98: Los Llanos de Aridane.
Closest hotel is Valle Aridane.
------------
Retrieving Results for Index 101: Ulcinj.
Closest hotel is Hotel Palata Venezia.
------------
Retrieving Results for Index 144: Vilhena.
Closest hotel is Hotel Mirage.
------------
Retrieving Results for Index 230: Alice Springs.
Closest hotel is Desert Palms Alice Springs.
------------
Retrieving Results for Index 261: Tezu.
Closest hotel is Circuit House.
------------
Retrieving Results for Index 304: Buckeye.
Closest hotel is Buckeye Motor Hotel.
------------
Retrieving Results for Index 347: Yulara.
Closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 363: Palazzolo Acreide.
Closest hotel is Feudo Bauly.
------------


In [25]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,56,Hilo,19.7297,-155.0900,77.29,96,0,1.01,US,1666265699,Hilo Hawaiian Hotel
98,98,Los Llanos de Aridane,28.6585,-17.9182,71.35,80,0,5.01,ES,1666265706,Valle Aridane
101,101,Ulcinj,41.9294,19.2244,72.99,44,0,5.64,ME,1666265706,Hotel Palata Venezia
144,144,Vilhena,-12.7406,-60.1458,73.62,88,0,5.75,BR,1666265711,Hotel Mirage
230,230,Alice Springs,-23.7000,133.8833,71.15,40,0,6.91,AU,1666265722,Desert Palms Alice Springs
261,261,Tezu,27.9167,96.1667,71.35,78,0,3.71,IN,1666265726,Circuit House
304,304,Buckeye,33.3703,-112.5838,75.83,47,0,0.00,US,1666265732,Days Inn by Wyndham Buckeye
347,347,Yulara,-25.2406,130.9889,76.51,27,0,7.18,AU,1666265738,Desert Gardens Hotel - Ayers Rock Resort
363,363,Palazzolo Acreide,37.0623,14.9035,71.69,41,0,4.56,IT,1666265670,Feudo Bauly


In [17]:
vaca_locations = hotel_df[["Lat", "Lng"]]

vaca_humidity = hotel_df["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
hotel_df_clean = hotel_df.dropna()
hotel_df_clean.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,56,Hilo,19.7297,-155.0900,77.29,96,0,1.01,US,1666265699,Hilo Hawaiian Hotel
98,98,Los Llanos de Aridane,28.6585,-17.9182,71.35,80,0,5.01,ES,1666265706,Valle Aridane
101,101,Ulcinj,41.9294,19.2244,72.99,44,0,5.64,ME,1666265706,Hotel Palata Venezia
144,144,Vilhena,-12.7406,-60.1458,73.62,88,0,5.75,BR,1666265711,Hotel Mirage
230,230,Alice Springs,-23.7000,133.8833,71.15,40,0,6.91,AU,1666265722,Desert Palms Alice Springs


In [19]:
dream_vacation= hotel_df_clean.set_index('City')
dream_vacation.head()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City,,,,,,,,,,
Hilo,56,19.7297,-155.0900,77.29,96,0,1.01,US,1666265699,Hilo Hawaiian Hotel
Los Llanos de Aridane,98,28.6585,-17.9182,71.35,80,0,5.01,ES,1666265706,Valle Aridane
Ulcinj,101,41.9294,19.2244,72.99,44,0,5.64,ME,1666265706,Hotel Palata Venezia
Vilhena,144,-12.7406,-60.1458,73.62,88,0,5.75,BR,1666265711,Hotel Mirage
Alice Springs,230,-23.7000,133.8833,71.15,40,0,6.91,AU,1666265722,Desert Palms Alice Springs


In [20]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in dream_vacation.iterrows()]
locations = dream_vacation[["Lat", "Lng"]]

In [21]:
import collections
collections.Iterable = collections.abc.Iterable

In [22]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 0.4)', scale=1,
    info_box_content=hotel_info
)
# Add the layer to the map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))